In [1]:
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt

In [ ]:
BATCH_SIZE = 640
EPOCHS = 10
LEARNING_RATE = 0.001

In [2]:
path = "../input/sudoku.csv"
data = pd.read_csv(path)
data = pd.DataFrame({"quizzes" : data["puzzle"], "solutions" : data["solution"]})
data.head()

,quizzes,solutions
0,0700000430400096108006349000940520003584600200...,6795182435437296188216349577943521863584617292...
1,3010865040465210705000000014008000020803479000...,3719865248465213795924738614638197522853479167...
2,0483015603600080909106700030200009355090102006...,7483915623652487919126754834217869355894132766...
3,0083170000042051090000400703271609049014500000...,2983176457642851391539462783271689549814537266...
4,0408906300001368208007405190004670524500207002...,1428956379751368248367425193984671524513287962...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000000 entries, 0 to 8999999
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   quizzes    object
 1   solutions  object
dtypes: object(2)
memory usage: 137.3+ MB


In [8]:
def create_model():
    model = Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(729))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))

    adam = keras.optimizers.Adam(lr=LEARNING_RATE)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

In [9]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 9, 9, 64)          640       
                                                                 
 batch_normalization (BatchN  (None, 9, 9, 64)         256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 batch_normalization_1 (Batc  (None, 9, 9, 64)         256       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 128)         8320      
                                                                 
 flatten (Flatten)           (None, 10368)             0

c:\Users\woute\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['quizzes'], data['solutions'], test_size=0.2, random_state=42)
del data

In [6]:
X_train, X_test, y_train, y_test = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [7]:
X_train = np.array([list(x) for x in X_train]).reshape(-1, 9, 9)
X_test = np.array([list(x) for x in X_test]).reshape(-1, 9, 9)
y_train = np.array([list(x) for x in y_train]).reshape(-1, 9, 9)
y_test = np.array([list(x) for x in y_test]).reshape(-1, 9, 9)

In [ ]:
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_test, y_test))